In [1]:
from vessel_classifier import VesselClassifier
import os
import PIL
import numpy as np
import sklearn
import matplotlib.pyplot as plt

In [2]:
# Load data
img_path = '../OSIRIX Images'
seg_path = '../Ground Truth'

def parse_seg(seg):
    seg = np.asarray(seg)
    seg[seg[...,3] == 0] = 0
    seg = seg[...,:3]
    seg[seg>0] = 1
    seg = np.squeeze(seg[...,0])
    return seg.astype(np.uint8)

images, segmentations = [], []

for imname in os.listdir(img_path):
    fname = os.path.join(img_path, imname)
    img = PIL.Image.open(fname).convert("L")
    img = np.asarray(img).astype(np.uint8)
    
    seg1_name = os.path.join(seg_path, "Observer 1", imname.replace(".jpg", ".png"))
    seg1 = PIL.Image.open(seg1_name) #.convert("L")
    seg2_name = os.path.join(seg_path, "Observer 2", imname.replace(".jpg", ".png"))
    seg2 = PIL.Image.open(seg2_name) #.convert("L")
    
    seg1, seg2 = parse_seg(seg1), parse_seg(seg2)
    seg = np.logical_or(seg1, seg2).astype(np.uint8)

    images.append(img)
    segmentations.append(seg)

In [3]:
from filter_extraction import *
from connectivity_features import *

In [4]:
feature_extractors = [
    lambda identity: np.expand_dims(identity, -1),
    hessian_features,
    surrounding_information,
    anisotropic_diffusion,
    morphology,
    kuwahara_features,
    light_sobel_filter,
    entropy_filters,
    gaussian_blurs
]

connectivity_extractors = [
    immediate_connectivity,
    radial_connectivity
]

In [9]:
class VesselClassifierWithPerfectSeeds(VesselClassifier):
    @staticmethod
    def _generate_init_mask(image_clf, n_seeds=1000):
        global images, segmentations
        
        init_mask = np.zeros_like(image_clf).flatten()
        
        for image, segmentation in zip(images, segmentations):
            if np.array_equal(image_clf, image):
                indices = np.argwhere(segmentation.flatten() > 0)
                indices = indices.flatten()
                np.random.shuffle(indices)
                init_mask[indices[:3]] = 1
                init_mask = init_mask.reshape(image_clf.shape)
                while np.sum(init_mask) < n_seeds:
                    init_mask = cv2.dilate(init_mask, np.ones((3, 3)))
                    init_mask[segmentation == 0] = 0
                init_mask = cv2.erode(init_mask, np.ones((3, 3)))
                return init_mask
        raise ValueError("Perfect mask not found")

In [10]:
vessel_classifier = VesselClassifierWithPerfectSeeds(
    feature_extractors,
    connectivity_extractors,
    verbose=2
)

In [7]:
params = {
    'base_estimator': [],
    'fit_mode': ['static'],
    'pca_dim': list(range(5, 25, 5)) + [0],
    'prediction_proba_threshold': [0.5, 0.7, 0.9],
    'connectivity_reduction_method': ['binary', 'normalised']
}

for n_estimators in range(2, 202, 25):
    for max_depth in list(range(10, 110, 20)) + [None]:
        params['base_estimator'].append(
            sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
        )

def label_pixelwise_f1(y_true, y_pred):
    y_true_flat = []
    y_pred_flat = []
    true = 0
    false = 0
    for label_true, label_pred in zip(y_true, y_pred):
        y_true_flat.extend(list(label_true.reshape(-1)))
        y_pred_flat.extend(list(label_pred.reshape(-1)))
    return sklearn.metrics.f1_score(y_true_flat, y_pred_flat)
        
search = sklearn.model_selection.GridSearchCV(
    scoring=sklearn.metrics.make_scorer(label_pixelwise_f1),
    verbose=3,
    n_jobs=8,
    cv=3,
    param_grid=params,
    estimator=vessel_classifier
)        

In [8]:
search.fit(images, segmentations)

Fitting 3 folds for each of 1440 candidates, totalling 4320 fits
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 2/3] END base_estimator=RandomForestClassifier(max_depth=10, n_estimators=2), connectivity_reduction_method=binary, fit_mode=static, pca_dim=5, prediction_proba_threshold=0.7;, score=0.049 total time=  43.5s
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 3/3] END base_estimator=RandomForestClassifier(max_depth=10, n_estimators=2), connectivity_reduction_method=binary, fit_mode=static, pca_dim=5, prediction_prob

Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
Starting segmentation for a new image
[CV 1/3] END base_estimator=RandomForestClassifier(max_depth=10, n_estimators=2), connectivity_reduction_method=binary, fit_mode=static, pca_dim=5, prediction_proba_threshold=0.9;, score=0.101 total time= 1.3min
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
Starting segmentation for a new image
[CV 1/3] END base_estimator=RandomForestClassifier(max_depth=10, n_estimators=2), connectivity_reduction_method=binary, fit_mode=static, pca_dim=10, prediction_

Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
Starting segmentation for a new image
[CV 1/3] END base_estimator=RandomForestClassifier(max_depth=10, n_estimators=2), connectivity_reduction_method=binary, fit_mode=static, pca_dim=5, prediction_proba_threshold=0.7;, score=0.300 total time= 2.7min
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 2/3] END base_estimator=RandomForestClassifier(max_depth=10, n_estimators=2), connectivity_reduction_method=binary, fit_mode=static, pca_dim=10, prediction_proba_threshold=0.9;, score=0.049 

Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 2/3] END base_estimator=RandomForestClassifier(max_depth=10, n_estimators=2), connectivity_reduction_method=binary, fit_mode=static, pca_dim=5, prediction_proba_threshold=0.9;, score=0.077 total time= 1.1min
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 2/3] END base_estimator=RandomForestClassifier(max_depth=10, n_estimators=2), connectivity_reduction_method=binary, fit_mode=static, pca_dim=10, prediction_proba_threshold=0.5;, score=0.365 total time= 3.6min
Processing inpu

Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 3/3] END base_estimator=RandomForestClassifier(max_depth=10, n_estimators=2), connectivity_reduction_method=binary, fit_mode=static, pca_dim=5, prediction_proba_threshold=0.5;, score=0.059 total time=  55.7s
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
Starting segmentation for a new image
[CV 1/3] END base_estimator=RandomForestClassifier(max_depth=10, n_estimators=2), connectivity_reduction_method=binary, fit_mode=static, pca_dim=10, prediction_proba_threshold=0.5;, score=0.553 

...

Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 2/3] END base_estimator=RandomForestClassifier(max_depth=90, n_estimators=177), connectivity_reduction_method=normalised, fit_mode=static, pca_dim=15, prediction_proba_threshold=0.7;, score=0.287 total time=26.7min
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 3/3] END base_estimator=RandomForestClassifier(max_depth=90, n_estimators=177), connectivity_reduction_method=normalised, fit_mode=static, pca_dim=20, prediction_proba_threshold=0.5;, score=0.482 total time=36.8min
Pr

Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 3/3] END base_estimator=RandomForestClassifier(n_estimators=177), connectivity_reduction_method=binary, fit_mode=static, pca_dim=0, prediction_proba_threshold=0.5;, score=0.047 total time=11.0min
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
Starting segmentation for a new image
[CV 1/3] END base_estimator=RandomForestClassifier(n_estimators=177), connectivity_reduction_method=normalised, fit_mode=static, pca_dim=5, prediction_proba_threshold=0.5;, score=0.473 total time=17.7min
Processing input 

Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 2/3] END base_estimator=RandomForestClassifier(n_estimators=177), connectivity_reduction_method=normalised, fit_mode=static, pca_dim=15, prediction_proba_threshold=0.7;, score=0.276 total time=26.7min
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
Starting segmentation for a new image
[CV 1/3] END base_estimator=RandomForestClassifier(n_estimators=177), connectivity_reduction_method=normalised, fit_mode=static, pca_dim=20, prediction_proba_threshold=0.7;, score=0.449 total time=28.2

Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
Starting segmentation for a new image
[CV 1/3] END base_estimator=RandomForestClassifier(n_estimators=177), connectivity_reduction_method=normalised, fit_mode=static, pca_dim=10, prediction_proba_threshold=0.5;, score=0.607 total time=22.5min
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 3/3] END base_estimator=RandomForestClassifier(n_estimators=177), connectivity_reduction_method=normalised, fit_mode=static, pca_dim=10, prediction_proba_threshold=0.9;, score=0.118 total time=22.1

GridSearchCV(cv=3,
             estimator=VesselClassifierWithPerfectSeeds(connectivity_extractors=[<function ensure_flattened.<locals>.filter_expanded_dim at 0x7f97b5db2f70>,
                                                                                 <function ensure_flattened.<locals>.filter_expanded_dim at 0x7f97b5d750d0>],
                                                        feature_extractors=[<function <lambda> at 0x7f97b5d75700>,
                                                                            <function ensure_flattened.<locals>.filter_expanded_dim at 0x7f97b5...
                                            RandomForestClassifier(max_depth=70,
                                                                   n_estimators=102),
                                            RandomForestClassifier(max_depth=90,
                                                                   n_estimators=102),
                                            RandomForestClassifier(n_e

In [9]:
# test_image_dir = '../nature_images'

# test_images = []

# for imname in os.listdir(test_image_dir):
#     fname = os.path.join(test_image_dir, imname)
#     img = PIL.Image.open(fname).convert("L")
#     img = np.asarray(img).astype(np.uint8)
    
#     test_images.append(img)

In [10]:
# for idx, test_image in enumerate(test_images):
#     pred = vessel_classifier.predict([test_image], 0.5)
#     plt.subplot(1, 2, 1)
#     plt.imshow(test_image, plt.cm.gray)
#     plt.axis("off")
#     plt.subplot(1, 2, 2)
#     plt.imshow(pred[0], plt.cm.gray)
#     plt.axis("off")
#     plt.tight_layout()
#     plt.savefig(os.path.join('../nature_output', '{}.png'.format(idx)), dpi=250)
#     plt.close()

In [11]:
try:
    from sklearn.externals import joblib
    joblib.dump(search.best_estimator_, 'best_estimator.pkl')
except:
    pass

In [12]:
import pickle
try:
    pickle.dump(search, 'search.pickle')
except:
    pass

Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 3/3] END base_estimator=RandomForestClassifier(n_estimators=177), connectivity_reduction_method=normalised, fit_mode=static, pca_dim=15, prediction_proba_threshold=0.7;, score=0.254 total time=26.1min
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Fitting PCA...
Fitting classifier...
GENERATING INIT MASK
GENERATING INIT MASK
Starting segmentation for a new image
Starting segmentation for a new image
[CV 3/3] END base_estimator=RandomForestClassifier(n_estimators=177), connectivity_reduction_method=normalised, fit_mode=static, pca_dim=20, prediction_proba_threshold=0.5;, score=0.500 total time=37.8min
Processing input image 0
Proce

In [19]:
search.best_estimator_

VesselClassifierWithPerfectSeeds(base_estimator=RandomForestClassifier(max_depth=90,
                                                                       n_estimators=27),
                                 connectivity_extractors=[<function ensure_flattened.<locals>.filter_expanded_dim at 0x7f97b5db2f70>,
                                                          <function ensure_flattened.<locals>.filter_expanded_dim at 0x7f97b5d750d0>],
                                 feature_extractors=[<function <lambda> at 0x7f97b5d75700>,
                                                     <function ensure_flattene...
                                                     <function ensure_flattened.<locals>.filter_expanded_dim at 0x7f97b5db29d0>,
                                                     <function ensure_flattened.<locals>.filter_expanded_dim at 0x7f97b5db2af0>,
                                                     <function ensure_flattened.<locals>.filter_expanded_dim at 0x7f97b5db2c1

In [20]:
params_specific = {
    'base_estimator': [],
    'fit_mode': ['static'],
    'pca_dim': [19, 20, 21],
    'prediction_proba_threshold': [0.5],
    'connectivity_reduction_method': ['binary', 'normalised']
}

for n_estimators in range(90, 110, 3):
    for max_depth in list(range(25, 35, 2)) + [None]:
        params_specific['base_estimator'].append(
            sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
        )

In [21]:
search_specific = sklearn.model_selection.GridSearchCV(
    scoring=sklearn.metrics.make_scorer(label_pixelwise_f1),
    verbose=3,
    n_jobs=8,
    cv=3,
    param_grid=params_specific,
    estimator=vessel_classifier
)

In [ ]:
search_specific.fit(images, segmentations)

Fitting 3 folds for each of 252 candidates, totalling 756 fits


In [23]:
search_specific.best_score_

0.6750873307389683

In [25]:
search_specific.best_params_

{'base_estimator': RandomForestClassifier(max_depth=29, n_estimators=108),
 'connectivity_reduction_method': 'binary',
 'fit_mode': 'static',
 'pca_dim': 19,
 'prediction_proba_threshold': 0.5}

In [47]:
nature_classifier = VesselClassifier(
    feature_extractors,
    connectivity_extractors,
    base_estimator = sklearn.ensemble.RandomForestClassifier(
        max_depth=29, n_estimators=100, n_jobs=12
    ),
    connectivity_reduction_method='binary',
    fit_mode='static',
    pca_dim=19,
    prediction_proba_threshold=0.5
)

In [48]:
nature_classifier.fit(images, segmentations)

In [49]:
test_image_dir = '../nature_images'

test_images = []

for imname in os.listdir(test_image_dir):
    fname = os.path.join(test_image_dir, imname)
    img = PIL.Image.open(fname).convert("L")
    img = np.asarray(img).astype(np.uint8)
    
    test_images.append(img)

In [ ]:
for idx, test_image in enumerate(test_images):
    pred = nature_classifier.predict([test_image], 0.6)
    plt.subplot(1, 2, 1)
    plt.imshow(test_image, plt.cm.gray)
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.imshow(pred[0], plt.cm.gray)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(os.path.join('../nature_output', '{}.png'.format(idx)), dpi=250)
    plt.close()

In [7]:
def label_pixelwise_accuracy(y_true, y_pred):
    y_true_flat = []
    y_pred_flat = []
    true = 0
    false = 0
    for label_true, label_pred in zip(y_true, y_pred):
        y_true_flat.extend(list(label_true.reshape(-1)))
        y_pred_flat.extend(list(label_pred.reshape(-1)))
    return sklearn.metrics.accuracy_score(y_true_flat, y_pred_flat)

In [8]:
params_accuracy = {
    'base_estimator': [sklearn.ensemble.RandomForestClassifier(n_estimators=100, max_depth=27)],
    'fit_mode': ['static'],
    'pca_dim': [19],
    'prediction_proba_threshold': [0.5],
    'connectivity_reduction_method': ['binary']
}


In [14]:
search_accuracy = sklearn.model_selection.GridSearchCV(
    scoring=sklearn.metrics.make_scorer(label_pixelwise_accuracy),
    verbose=3,
    n_jobs=8,
    cv=sklearn.model_selection.LeaveOneOut(),
    param_grid=params_accuracy,
    estimator=vessel_classifier
)

In [15]:
search_accuracy.fit(images, segmentations)

Fitting 7 folds for each of 1 candidates, totalling 7 fits
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Processing input image 5
Processing input image 6
Fitting PCA...
Fitting classifier...
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Processing input image 5
Fitting PCA...
Fitting classifier...
Starting segmentation for a new image
[CV 5/7] END base_estimator=RandomForestClassifier(max_depth=27), connectivity_reduction_method=binary, fit_mode=static, pca_dim=19, prediction_proba_threshold=0.5;, score=0.904 total time=12.2min
Processing input image 0
Processing input image 1
Processing input image 2
Processing input image 3
Processing input image 4
Processing input image 5
Fitting PCA...
Fitting classifier...
Starting segmentation for a new image
[CV 7/7] END base_estimator=RandomForestClassifier(max_depth=27), connectivity_red

GridSearchCV(cv=LeaveOneOut(),
             estimator=VesselClassifierWithPerfectSeeds(connectivity_extractors=[<function ensure_flattened.<locals>.filter_expanded_dim at 0x7f9ddc33caf0>,
                                                                                 <function ensure_flattened.<locals>.filter_expanded_dim at 0x7f9ddc33cc10>],
                                                        feature_extractors=[<function <lambda> at 0x7f9ddc2be040>,
                                                                            <function ensure_flattened.<locals>.filter_expanded_d...
                                                                            <function ensure_flattened.<locals>.filter_expanded_dim at 0x7f9ddc32ef70>,
                                                                            <function ensure_flattened.<locals>.filter_expanded_dim at 0x7f9ddc32ee50>],
                                                        verbose=2),
             n_jobs=8,
          

In [16]:
search_accuracy.best_score_

0.9397899082728794